In [1]:
#******LESSON 7 IN spai CHALLENGE******

In [3]:
#PRACTICE CODES ON SYFT

In [5]:
import torch as th
x = th.Tensor([1,2,3,4,5])
x
y = x + x
y


tensor([ 2.,  4.,  6.,  8., 10.])

In [ ]:
import syft as sy
hook = sy.TorchHook(th)
th.tensor([1,2,3,4,5]) #DEMO


In [ ]:
#Create a pretend machine and a pretend worker for nw
bob = sy.VirtualWorker(hook, id="bob")
bob._objects #workers are sets of objects..output is {}, cuz bob doesnt have any stored or received value

In [ ]:
x=th.tensor([1,2,3,4,5])
x=x.send(bob)


In [ ]:
bob._objects #gvs output similar to {63827205014: tensor([1,2,3,4,5])}
#it is proof that original tensor was sent to bob. we will be returned with a pointer to the remote object. print x nd check

x 
#output is of form (Wrapper) > [PointerTensor | me:93174343354 -> bob:63827205014].x is a tensor. however instead of local
#execution, each cmd is serialized to a simple JSON or tuple format sent to bob, he executes it on our behalf and send back the 
#pointer to the new object.

x.location # can also do == bob as test condition or so
#output similar to <VirtualWorker id:bob #tensors:1>

x.id_at_location #63827205014
x.id #93174343354
x.owner #<VirtualWorker id:me #tensors:0>


In [ ]:
x=x.get()
x #tensor([1,2,3,4,5])
bob._objects #{} will be the output....

In [ ]:
#Adding another VirtualWorker
alice = sy.VirtualWorker(hook, id="alice")
alice._objects #output is {} as of nw

In [ ]:
x=th.tensor([1,2,3,4,5])
x = x.send(bob, alice)
#we can also use x=th.tensor([1,2,3,4,5]).send(bob, alice)

In [ ]:
x #here x is a multipointerTensor with each sender having a PointerTensor

In [ ]:
x.child.child # returns a dictionary of sub PointerTensors of the MultiPointer
x.get() # returns 2 copies of the tensor sent cuz we hv 2 workers

x.get(sum_results = True) # returns a tensor ([2,4,6,8,10])

In [ ]:
#***INTRODUCING REMOTE ARITHMETIC***

In [ ]:
x=th.tensor([1,2,3,4,5]).send(bob)
y=th.tensor([1,1,1,1,1]).send(bob)

x #diffrnt me,bob value from y. 

y

z=x+y #returns me nd bob values equal to each other
z

z=z.get()
z #returns ([2,3,4,5,6]) i.e. addition of the two msg tensors

#we can also use z=th.add(x,y) z z=z.get() z to get the same output as abv...this is using torch funcn ashte

In [ ]:
x=th.tensor([1,2,3,4,5], requires_grad=True).send(bob)
y=th.tensor([1,1,1,1,1], requires_grad=True).send(bob)

In [ ]:
z=(x+y).sum()
z.backward() #back propagation

In [ ]:
x=x.get()
x #prints [1., 2., 3., 4., 5.]

x.grad #prints [1., 1., 1., 1., 1.,]

In [ ]:
#***LEARN A SIMPLE LINEAR MODEL WHICH IS ON SOME OTHER DEVICE***

In [ ]:
input = th.tensor([[1.,1], [1,0], [0,1], [0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]]).send(bob)

input #PointerTensor output
target #PointerTensor output

weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

for i in range(10):
    preds = input.nn(weights)
    loss = ((pred-target)**2).sum() #sum of mean squared error
    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0 #multiplying by zero so that they dont get accumulated over time

    print(loss.get().data) #prints tensor(2.)



In [ ]:
#***GARBAGE COLLECTION AND COMMON ERRORS***

In [ ]:
bob = bob.clear_objects()
bob._objects #output is {}

x=th.tensor([1,2,3,4,5]).send(bob)
bob._objects #will contain the tensor sent to bob

del x
bob._objects #output is {}

#this is controlled by a special attribute i.e. x.child.garbage_collect_data which is set to True by default.what this attribute
#does is it sends a message to the Virtualworker, here, Bob to delete the tensor which was sent by us cuz we have deleted the 
#"me" pointer.

#GOTCHA IN JUPYTER NOTEBOOK...this happens with reference to garbage collection of some pointers
x=th.tensor([1,2,3,4,5]).send(bob)
bob._objects #prints the output
x="asdf"
bob._objects #output is {}

#DOESNT HAPPEN EVERYTIME
x=th.tensor([1,2,3,4,5]).send(bob)
bob._objects #prints output
x="asdf"
bob._objects #prints output
del x
bob._objects #prints output

bob=bob.clear_objects()
bob._objects #output is {}



In [ ]:
#***TOY FEDERATED LEARNING***

In [ ]:
from torch import nn, optim

#Create a toy dataset
input = th.tensor([[1.,1], [1,0], [0,1], [0,0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]])

#Create toy model
model=nn.Linear(2,1)

opt=optim.SGD(param=model.parameters(), lr=0.1) #stochastic gradient descent

def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred=model(data)

        loss = ((pred-target)**2).sum()
        #loss #output is tensor(0.4472, gradfunction=<SumBackward0>)

        loss.backward()
        opt.step()

        print(loss.data) #output is 0.4472

In [ ]:
#implementing federated learning for above model

In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

datasets = [(data_bob, target_bob), (data_alice, target_alice)] #tuples in list

def trains(iterations=20):
    model = nn.Linear(2,1)
    opt=optim.SGD(param=model.parameters(), lr=0.1)

    for iter in range(iterations):
        for _data, _target in datasets:
            #_data #it is a PointerTensor from me to Bob
            #_data.location #output is VirtualWorker with id Bob

            """Send model to remote device"""
            #list(model.parameters()) #output is a list of weight nd bias values
            model=model.send(_data.location)
            #list(model.parameters()) #output is a list of pointers to data present elsewhere i.e. Bob's machine

            """Do normal training"""
            opt.zero_grad()
            pred=model(_data)
            loss = ((pred-_target)**2).sum()
            #loss #output is tensor(0.4472, gradfunction=<SumBackward0>)
            loss.backward()
            opt.step()

            """Get smarter model back (to the cloud maybe!)"""
            model=model.get()
            print(loss.get())

#when bob's device trains only for 1 iteration            
"""Difference between model sent and model got back can be used in reverse engineering to try and deduce data on the device"""
#1 way to overcome : Train for many iterations on bob's device....
#2 way to overcome : instead of bringing model back to us directly and then sending to alice, train the model parallely on many
#                   workers so tht many changes are made and then we cant reverse engineer as to wht is the data nd whose is it?
#                   we get back the average of multiple people's models!



In [ ]:
#ADVANCED REMOTE EXECUTION, TRAINING MODEL PARALLELY ON MULTIPLE REMOTE DEVICES

In [ ]:
#clearing previously stored objects
bob.clear_objects()
alice.clear_objects()

x=th.tensor([1,2,3,4,5]).send(bob) #x is wrapper to pointertensor
bob._objects #tensor([1,2,3,4,5]) with a pointer id

x = x.send(alice) #x will point to alice now
alice._objects #wrapper to pointertensor but from bob to alice i.e. alice -> bob

y = x + x #when we need to access the tensor[1,2,3,4,5], we use x which points to alice which uses a pointer to bob to access it
y #output is a wrapper to pointertensor from me to alice 

bob._objects #contains 2 tensors [1,2,3,4,5] and [2,4,6,8,10]
alice._objects #contains 2 pointers to the 2 tensors with bob

#Bob and Alice are owners of the data. If either one of them doesnt allow us to modify data, for any reason, then we cannot 
#access it at all. The data has a dependency chain...from me to alice to bob. if alice forwards it to bob nd bob allows, then ok
#if not or if alice doesnt forward at all, den not possible to access data.

#create a 3rd worker and create 2 different dependency chains
Jon=sy.VirtualWorker(hook, id="Jon")

bob.clear_objects()
alice.clear_objects()

x=th.tensor([1,2,3,4,5]).send(bob).send(alice)
y=th.tensor([1,2,3,4,5]).send(bob).send(jon)

bob._objects #will have two tensors [1,2,3,4,5] and [1,2,3,4,5]
alice._objects #will have a pointertensor from alice to bob
jon._objects #will have a pointertensor from jon to bob

# we cannot have an operation involving these 2 tensors as their dependency chains r different
#z = x + y #returns a TensorNotCollocated Exception
    
#then how do we get this data???
x = x.get() #returns a pointer to the data from the pointer to alice's machine
x #pointertensor from me to bob

alice._objects #{}. she no longer has the pointer cuz she sent it to us

x = x.get()
x #output is the tensor [1,2,3,4,5]

bob._objects #{}, cuz he sent the tensor to us


#GARBAGE COLLECTION

bob.clear_objects()
alice.clear_objects()

x=th.tensor([1,2,3,4,5]).send(bob).send(alice)

bob._objects #has the tensor [1,2,3,4,5]
alice._objects #pointertensor from alice to bob

del x 
bob._objects #{}
alice._objects #{}

"""Garbage collected the whole chain since we deleted the pointer, it shud delete the data that it was pointing to"""

In [ ]:
#***POINTER CHAIN OPERATIONS***

In [ ]:
bob.clear_objects()
alice.clear_objects()

x=th.tensor([1,2,3,4,5]).send(bob).send(alice)

bob._objects #has the tensor [1,2,3,4,5]
alice._objects #pointertensor from alice to bob

x.remote_get() #using get returns a pointer from alice to bob, but when we use remote_get(),it goes all the way down the pointer
#chain and removes data from der...

x #will remain a pointer from me to alice
bob._objects #{}
alice._objects #tensor[1,2,3,4,5]

"""in fact what we try to tell alice is 'Hey, use get() on ur pointer and pull data from bob to urself!' """

x.move(bob) #move is a convenience operator..it actually takes the pointer and calls remote_get() on it internally
x #it will now be a pointer to bob and not alice!

bob._objects #tensor[1,2,3,4,5]
alice._objects #tensor[1,2,3,4,5]. this is a garbage collection issue as we still have a pointer to it.

#Rectified as following example
bob.clear_objects()
alice.clear_objects()

x=th.tensor([1,2,3,4,5]).send(bob)

x.move(alice)

alice._objects #will have tensor
bob._objects #will be empty {}

